In [ ]:
import os


account_id = os.getenv("AWS_ACCOUNT")
image_config = {'RepositoryAccessMode': 'Platform'}
primary_container = {
    'ContainerHostname': 'ModelContainer',
    'Image': f'{account_id}.dkr.ecr.us-west-2.amazonaws.com/whisper-gpu:latest',
    'ImageConfig': image_config,
    'ModelDataUrl': 's3://sagemaker-us-west-2-{account_id}/whisper-gpu-deploy/model/model.tar.gz'
}
role_name = "SageMakerExecutionRole"
model_name="whisper-gpu-model"
execution_role_arn = 'arn:aws:iam::{account_id}:role/SageMakerExecutionRole'

In [ ]:
import boto3
import json

# Initialize the IAM client
iam_client = boto3.client('iam')

sagemaker_policy_arn = 'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'

# Define the trust policy for SageMaker
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Check if the IAM role already exists
try:
    existing_role = iam_client.get_role(RoleName=role_name)
    role_arn = existing_role['Role']['Arn']

    # Update the role's trust policy to allow SageMaker
    iam_client.update_assume_role_policy(
        RoleName=role_name,
        PolicyDocument=json.dumps(trust_policy)
    )

    print(f"IAM Role '{role_name}' already exists. Updated trust policy.")
except iam_client.exceptions.NoSuchEntityException:
    # Create the IAM role if it doesn't exist
    role_description = "IAM role for SageMaker"
    response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description=role_description
    )

    role_arn = response['Role']['Arn']

    print(f"IAM Role '{role_name}' created with trust policy for SageMaker.")

# Attach the AmazonSageMakerFullAccess policy to the IAM role
iam_client.attach_role_policy(
    PolicyArn=sagemaker_policy_arn,
    RoleName=role_name
)

print(f"Attached AmazonSageMakerFullAccess policy to IAM Role '{role_name}'.")

print(f"IAM Role ARN: {role_arn}")

In [ ]:
import boto3
sm = boto3.client('sagemaker')

try:
    resp = sm.create_model(
        ModelName=model_name,
        PrimaryContainer=primary_container,
        ExecutionRoleArn=execution_role_arn
    )
except Exception as e:
    print(f'error calling CreateModel operation: {e}')
else:
    print(resp)

In [ ]:
endpoint_config_name = 'whisper-gpu-endpoint-config'
sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'WhisperGpu',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.p3.8xlarge'
        },
    ],
)

In [ ]:
endpoint_name = 'whisper-gpu-endpoint'
sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)
sm.describe_endpoint(EndpointName=endpoint_name)